# Internet of Things:  Fingerprint Based Localisation

## 1 Objectives
In the previous lab we have learnt how to measure RSSI, and discussed prob-
lems on RSSI based ranging, especially in indoor environments. 

The noisy RSSI measurements and environment-dependent log-distance path loss model significantly hinder the use of RSSI based ranging for indoor localisation. In this lab, we are going to work on a popular and powerful indoor positioning technique: fingerprint based localisation.

Please send your results (PDF from Jupyter) to nhat.pham@cs.ox.ac.uk by Monday.

## 2 WiFi fingerprinting
### 2.1 Datasets
You were given two datasets, namely **set1** and **set2**, containing WiFi RSSI data for 3 and 5 access points (APs), respectively. Inside each set you will find a **APloc.txt** file, which gives the ground truth positions for all APs in that dataset. 

Inside the **wifiData** folder of each dataset you will find a number of **Wifi_xxx.txt** files, each of which containing the *(x, y)* position where the measurements were taken (first line) followed by a series of RSSI measurements.

The **testWifiData** folder, which has the same structure as the **wifiData** folder, contains only RSSI data and will be used for testing your positioning system.

### 2.2 Gaussian approximation

Using the provided dataset **set1**, plot the RSSI histogram of **AP1** at 3 different locations. Use a Gaussian distribution to model the RSSI for the 3 locations above and compare it with the histogram. Is the Gaussian distribution a good choice? What are the pros and cons of using a Gaussian distribution to approximate the RSSI distribution in this scenario? Please include the figures in the report and answer the questions.

In [ ]:
# Generate plots here.

*Answer the previous questions* **here**.

### 2.3 WiFi trajectory
Based on the training data in **set1/wifiData/**, please complete the initial code
below to implement a fingerprint based localisation system.

Test the resulting algorithm on the testing data **set1/testWifiData**. 

1. Plot the estimated localisation trajectory against the ground truth and 
2. Plot the cumulative distribution function (CDF) of the localisation errors. 
3. Explain your algorithm and the results.

In [ ]:
########################################################
## OFFLINE PHASE: Choose dataset
########################################################
import os
import numpy as np
from scipy.stats import norm
%matplotlib notebook
import matplotlib.pyplot as plt

# Read data from the dataset
dataset_name = os.path.join('wifi', 'set1') #CHANGE HERE IF USING set2!!!



In [ ]:
########################################################
## OFFLINE PHASE: Build radio map using training data
########################################################
# TODO 1st: build a radio map by using *rssi* and *loc_xy*
# [loc_x, loc_y, mu1, mu2, ...., muN, sigma1, sigma2, ....sigmaN], N is number of APs


# Read Access Points Locations 
ap_loc = []
with open(os.path.join(dataset_name, 'APloc.txt')) as f:
  for line in f:
    ap, x, y = line.split()
    ap_loc.append([float(x), float(y)])
ap_loc = np.array(ap_loc)

# Offline phase: build radio map by using training data and get file names
wifi_file_names = [filename for filename in os.listdir(os.path.join(dataset_name, 'wifiData') ) if filename.endswith('.txt') ]
wifi_file_names.sort()


# Read training data and build radio map
num_aps = len(ap_loc)
radio_map = np.zeros((len(wifi_file_names), 2 + num_aps * 2), dtype=np.float)
for measurement_idx, file_name in enumerate( wifi_file_names) :
    file_path = os.path.join(*(dataset_name, 'wifiData', file_name))
    
    with open(file_path) as measurements:
        loc_x, loc_y = measurements.readline().split()
        loc_x, loc_y = float(loc_x), float(loc_y)
        
        rssi_values = np.loadtxt(measurements.readlines(), delimiter=' ', usecols=(2), unpack=True)
        rssi_values = np.reshape(rssi_values, (-1, num_aps ))
        
        # Fit a normal distribution to the data:
        mu, std = np.apply_along_axis( norm.fit, 0, rssi_values)
        
        radio_map[measurement_idx] = np.concatenate(([loc_x, loc_y], mu, std), axis=0)

num_fingerprints = radio_map.shape[0]

In [ ]:
###############################################################
## ONLINE PHASE: Estimate path using radio map previously built 
###############################################################


## Get name test files
wifi_file_names = [filename for filename in os.listdir(os.path.join(dataset_name, 'testWifiData') ) if filename.endswith('.txt') ]
wifi_file_names.sort()
estimated_position = np.zeros((len(wifi_file_names), 2))
ground_truth = []

for measurement_idx, file_name in enumerate(wifi_file_names):
  file_path = os.path.join(*(dataset_name, 'testWifiData', file_name))
    
  with open(file_path) as measurements :
    gt_x, gt_y = measurements.readline().split()
    ground_truth.append([float(gt_x), float(gt_y)])
    
    
    rssi_values = np.loadtxt(measurements.readlines(), delimiter=' ', usecols=(2), unpack=True)
    rssi_values = np.reshape(rssi_values, (-1, num_aps ))
    rssi_mean   = np.apply_along_axis(np.mean, 0, rssi_values) # mean value 

    # TODO 2nd: calculate probability of current location for each radio map points
    log_prob = np.zeros((num_fingerprints, 1)) ;
    
    for fp_idx, finger_print in enumerate(radio_map) :
      # finger_print = [loc_x, loc_y, mu1, mu2, ...., muN, sigma1, sigma2, ....sigmaN]
      for ap_idx in range(num_aps):
        prob = norm( finger_print[2 + ap_idx] , finger_print[ 2 + num_aps + ap_idx ]).pdf(rssi_mean[ap_idx])                
        log_prob[fp_idx] += np.log(prob);
    
    
    # Locate center of mass of top candidate locations for continuous space estimator
    sorted_log_prob = np.sort(log_prob, axis=0)[::-1] # Descending order, change this if you want random fingerprints
    sorted_log_prob_idx  = np.argsort(log_prob, axis=0)[::-1] # Descending order, change this if you want random fingerprints

    num_fingerprints_used = num_fingerprints
    top_log_prob = sorted_log_prob[:num_fingerprints_used]
    top_log_prob_idx = sorted_log_prob_idx[:num_fingerprints_used,0]
    top_fingerprint_positions = radio_map[top_log_prob_idx[:],:2] # Get x,y from best lob_prob fingerprints 
    
    current_est_position = np.dot( np.exp( top_log_prob.transpose()), top_fingerprint_positions)/np.sum(np.exp(top_log_prob))
    #current_est_position = np.dot( top_log_prob.transpose(), top_fingerprint_positions)/np.sum(top_log_prob) #WRONG

    estimated_position[measurement_idx, : ] = current_est_position[0];

ground_truth = np.array(ground_truth)

/home/pedro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log


In [ ]:
###################
## PLOT TRAJECTORY 
###################
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(ap_loc[:,0], ap_loc[:,1], 'rs', markersize=2, label='Access Points');
ax.plot(ground_truth[:, 0],ground_truth[:, 1],  linewidth=2, markersize=2, label='Ground Truth')
ax.plot(estimated_position[:, 0],estimated_position[:, 1], '--', linewidth=1.2, label='Estimated Positions')
leg = ax.legend()
plt.title('Wifi Localisation')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.show()

In [ ]:
###################
## PLOT TRAJECTORY 
###################

# Calculate erros
loc_errors = np.sqrt(np.sum(np.multiply(ground_truth - estimated_position, ground_truth - estimated_position), axis=1) );

fig, ax = plt.subplots(figsize=(8, 4))
# Plot the cumulative histogram
n, bins, patches = ax.hist(loc_errors, bins=100, density=True, histtype='step',
                           cumulative=True, label='Empirical CDF')
# Tidy up the figure
ax.grid(True)
ax.legend(loc='right')
ax.set_title('Cumulative step histograms')
ax.set_xlabel('Distance Error (m)')
ax.set_yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
ax.set_ylabel('Probability')

plt.show()


*Explain your algorithm and the results* **here**.

### 2.4 OPTIONAL:
Your previous algorithm was trained and tested based on dataset **set1** that only used 3 WiFi APs. 

Repeat the process with dataset **set2** which contains 5 WiFi APs. Please explain how the number of APs affects the localisation accuracy. 

Plot the trajectories and CDF of localisation errors, and compare the results with your previous ones. Are the results expected? Discuss the reasons.

## 3 Magneto-inductive fingerprinting
As we saw in lectures, MI based techniques exploit a different physical model,
based on quasi-static magnetic fields to determine position. 
In undistorted environments (e.g. where there is not a large amount of ferromagnetic material)
the relationship between position/pose and the received channel matrix can be
determined using a closed form model. However, distorted environments can
be treated exactly the same as a WiFi fingerprinting problem. 

In this section of the lab, we will adapt the methods you have created for WiFi fingerprinting
and apply them to the problem of MI fingerprinting.

### 3.1 Datasets
Two MI transmitters (**TX_1** and **TX_2**) were placed in an environment at undisclosed locations and a number of fingerprints were taken at known positions in a horizontal plane (x, y) using an MI receiver, held at a fixed orientation during the experiment. 

For each transmitter, the receiver measures the 3 × 3 channel
matrix (hence 9 numbers in total: e.g. **TX_2_6** is the second row, third column from **TX_2**). 

You have been given two files, **mi_dataset_train.csv** and **mi_dataset_test.csv**. Use the training file to perform fingerprinting and the test file for measuring the performance of your technique.

### 3.2 Fingerprint creation
Plot the level surface of the Frobenius norm of the channel matrix for a single transmitter over the fingerprinting region. What do you observe? Now plot the
log of the Frobenius norm. 

**What do you observe now? What does this suggest
about a suitable transformation of the channel matrix or distance metric to use
in your fingerprinting approach?** 

Provide the answers to these questions and plots below.

*Provide answers* **here**.

In [ ]:
# Provide plots and code here

### 3.3  MI Localisation 
Adapt your fingerprinting method (HORUS or other e.g. Gaussian Process Regression) to use the MI training dataset. 

Describe your approach.

Using the test file, plot your trajectory against ground-truth and the error CDF. Can you still get good results if you only use a single transmitter? Discuss and include the plot in your report.
Fingerprinting is very tedious and boring. Examine what happens to the error CDF as the number of fingerprints used to train the model is decreased.

Briefly discuss the tradeoff between fingerprinting density and achievable accuracy. **BONUS: If you use three fingerprints as training data, can you make a simple triangulation approach work?**

Finally, in a few sentences, briefly compare the relative merits of WiFi
against MI fingerprinting.

In [ ]:
# Provide code here

In [ ]:
### Generate predictions 

## Get predicted positions based on two transmitters
x_pred = np.zeros((num_locs,))
y_pred = np.zeros((num_locs,)) 

## Using both sensors
x_pred_one_tx = np.zeros((num_locs,))
y_pred_one_tx = np.zeros((num_locs,)) 

#Using one sensor (RENAME) Maybe sparse?
x_pred_sparse = np.zeros((num_locs,))
y_pred_sparse = np.zeros((num_locs,)) 

# Also run predictions for sparse (and random) fingerprints
num_rand_fingerprinters = 40 # randomly sample 40 fingerprints
random_subset_of_fingerprints_idx = np.random.permutation(num_fingerprints)[:num_rand_fingerprinters] 

TX1train_sparse = TX1train[random_subset_of_fingerprints_idx,:] 
TX2train_sparse = TX2train[random_subset_of_fingerprints_idx,:]
x_sparse = x[random_subset_of_fingerprints_idx]
y_sparse = y[random_subset_of_fingerprints_idx]

# Vary the number of best fingerprints and see what happens. 
# In the sparse case, is it better to have few high-probability (num_best_signals) signals or many random (num_rand_fingerprinters)?
num_best_signals = 5

## Evaluate for each location
for i in range(num_locs):
    
    TX1t = TX1test[i,:]
    TX2t = TX2test[i,:]
    
    d1 = np.zeros((num_fingerprints,))
    d2 = np.zeros((num_fingerprints,))
    
    ### Calculate distance based on both sensors
    for fi in range(num_fingerprints):
        d1[fi] = np.sum((TX1train[fi,:] - TX1t)**2);
        d2[fi] = np.sum((TX2train[fi,:] - TX2t)**2);
        
    d = d1 + d2;
    
    ## Center of mass of top candidate locations for continuous space estimator based on two sensors
    sorted_idx = np.argsort(d);    
    top_ind = sorted_idx[:num_best_signals] # Still selecting 
    
    top_xs = x[top_ind]
    top_ys = y[top_ind]    
    top_sorted = 1./d[top_ind]
    
    est_x = np.dot(top_xs, top_sorted)/np.sum(top_sorted)
    est_y = np.dot(top_ys, top_sorted)/np.sum(top_sorted)

    x_pred[i] = est_x 
    y_pred[i] = est_y

    ## Center of mass of top candidate locations for continuous space based on one (1) transmitter
    sorted_idx = np.argsort(d2); #Or d1, but not the sum    
    top_ind = sorted_idx[:num_best_signals] # Still selecting 
    
    top_xs = x[top_ind]
    top_ys = y[top_ind]    
    top_sorted = 1./d[top_ind]
    
    est_x = np.dot(top_xs, top_sorted)/np.sum(top_sorted)
    est_y = np.dot(top_ys, top_sorted)/np.sum(top_sorted)
    x_pred_one_tx[i] = est_x
    y_pred_one_tx[i] = est_y

    ## Sparse predictions
    d  = d[random_subset_of_fingerprints_idx] 
    sorted_idx = np.argsort(d);    
    top_ind = sorted_idx[:num_best_signals] # Still selecting 
    
    top_xs = x_sparse[top_ind];
    top_ys = y_sparse[top_ind];
    
    top_sorted = 1./top_sorted;
    est_x = np.dot(top_xs, top_sorted)/sum(top_sorted);
    est_y = np.dot(top_ys, top_sorted)/sum(top_sorted);
    
    x_pred_sparse[i] = est_x 
    y_pred_sparse[i] = est_y

In [ ]:
### Plot norms for test set
fig, axs = plt.subplots(1,2, figsize=(12, 6))
fig.suptitle('Testing Data')

# TX1test_norms
h1 = axs[0].scatter(x_test, y_test, s = 10, c = TX1test_norms)
axs[0].set_xticks(np.linspace(0, 16, 9,  endpoint=True))
axs[0].set_yticks(np.linspace(0, 16, 9,  endpoint=True))
axs[0].set_title('TX1test Frobenius norm')
axs[0].set_xlabel('x [m]') #ugly
axs[0].set_ylabel('y [m]')
axs[0].axis('equal')
axs[0].plot(x_pred, y_pred, label='Two transmitters')
axs[0].plot(x_pred_one_tx, y_pred_one_tx, 'r--', label='One transmitters')
axs[0].plot(x_pred_sparse, y_pred_sparse, 'g--', label='Random fingerprints')
leg = axs[0].legend()
plt.colorbar(h,ax=axs[0])


## Plot CDF
loc_errors = (x_test - x_pred)**2 + (y_test - y_pred)**2
loc_errors = np.sqrt(loc_errors)

loc_errors_one_tx = (x_test - x_pred_one_tx)**2 + (y_test - y_pred_one_tx)**2
loc_errors_one_tx = np.sqrt(loc_errors_one_tx)

loc_errors_sparse = (x_test - x_pred_sparse)**2 + (y_test - y_pred_sparse)**2
loc_errors_sparse = np.sqrt(loc_errors_sparse)

# Plot the cumulative histogram
n, bins, patches = axs[1].hist(loc_errors, bins=100, density=True, histtype='step',
                           cumulative=True, label='Two Transmitter')

n, bins, patches = axs[1].hist(loc_errors_one_tx, bins=100, density=True, histtype='step',
                           cumulative=True, label='One Transmitter')

n, bins, patches = axs[1].hist(loc_errors_sparse, bins=100, density=True, histtype='step',
                           cumulative=True, label='Sparse')

# Tidy up the figure
axs[1].grid(True)
axs[1].legend(loc='right')

axs[1].set_title('Cumulative step histograms')
axs[1].set_xlabel('Distance Error (m)')
axs[1].set_yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
axs[1].set_ylabel('Probability')

plt.show()



*Provide answers* **here**.